References
- https://github.com/nassarhuda/JuliaTutorials
- https://github.com/PacktPublishing/Hands-on-Design-Patterns-and-Best-Practices-with-Julia/
- 

## Performance of global constant vs. global variable

Global variables hurts performance because compiler cannot generate
efficient code. 

In [1]:
using BenchmarkTools

# using global variable
variable = 10
function add_using_global_variable(x)
    return x + variable
end

@btime add_using_global_variable(10);

  21.924 ns (0 allocations: 0 bytes)


In [2]:
function add_using_function_arg(x, y)
    return x + y
end

@btime add_using_function_arg(10, variable);

  17.232 ns (0 allocations: 0 bytes)


In [5]:
@code_llvm add_using_function_arg(10, 10)

;  @ In[4]:1 within `add_using_function_arg`
define i64 @julia_add_using_function_arg_2960(i64 signext %0, i64 signext %1) #0 {
top:
;  @ In[4]:2 within `add_using_function_arg`
; ┌ @ int.jl:87 within `+`
   %2 = add i64 %1, %0
; └
  ret i64 %2
}


In [6]:
@code_llvm add_using_global_variable(10)

;  @ In[3]:5 within `add_using_global_variable`
define nonnull {}* @julia_add_using_global_variable_2983(i64 signext %0) #0 {
top:
  %1 = alloca [2 x {}*], align 8
  %gcframe2 = alloca [4 x {}*], align 16
  %gcframe2.sub = getelementptr inbounds [4 x {}*], [4 x {}*]* %gcframe2, i64 0, i64 0
  %.sub = getelementptr inbounds [2 x {}*], [2 x {}*]* %1, i64 0, i64 0
  %2 = bitcast [4 x {}*]* %gcframe2 to i8*
  call void @llvm.memset.p0i8.i32(i8* noundef nonnull align 16 dereferenceable(32) %2, i8 0, i32 32, i1 false)
  %3 = call {}*** inttoptr (i64 140703197364181 to {}*** (i64)*)(i64 260) #3
;  @ In[3]:6 within `add_using_global_variable`
  %4 = bitcast [4 x {}*]* %gcframe2 to i64*
  store i64 8, i64* %4, align 16
  %5 = load {}**, {}*** %3, align 8
  %6 = getelementptr inbounds [4 x {}*], [4 x {}*]* %gcframe2, i64 0, i64 1
  %7 = bitcast {}** %6 to {}***
  store {}** %5, {}*** %7, align 8
  %8 = bitcast {}*** %3 to {}***
  store {}** %gcframe2.sub, {}*** %8, align 8
  %9 = load atomic {}*

In [7]:
const constant = 10
function add_using_global_constant(x)
    return constant + x
end

@btime add_using_global_constant(10);

  1.656 ns (0 allocations: 0 bytes)


In [8]:
@code_llvm add_using_global_constant(10)

;  @ In[7]:2 within `add_using_global_constant`
define i64 @julia_add_using_global_constant_2989(i64 signext %0) #0 {
top:
;  @ In[7]:3 within `add_using_global_constant`
; ┌ @ int.jl:87 within `+`
   %1 = add i64 %0, 10
; └
  ret i64 %1
}


what if variable is annotated with type?

In [10]:
variable_typed::Int = 10

10

In [11]:
function add_using_global_variable_typed(x)
    return x + variable_typed::Int
end

add_using_global_variable_typed (generic function with 1 method)

In [12]:
@btime add_using_global_variable_typed(10);

  2.851 ns (0 allocations: 0 bytes)


In [17]:
@code_llvm add_using_global_variable_typed(10)

;  @ In[11]:1 within `add_using_global_variable_typed`
define i64 @julia_add_using_global_variable_typed_3265(i64 signext %0) #0 {
top:
;  @ In[11]:2 within `add_using_global_variable_typed`
  %1 = load atomic i64*, i64** inttoptr (i64 4598536472 to i64**) unordered, align 8
; ┌ @ int.jl:87 within `+`
   %2 = load i64, i64* %1, align 8
   %3 = add i64 %2, %0
; └
  ret i64 %3
}


## Constant folding

In [18]:
function constant_folding_example()
    a = 2 * 3
    b = a + 1
    return b > 1 ? 10 : 20
end
constant_folding_example()
@code_typed constant_folding_example()

CodeInfo(
1 ─     return 10
) => Int64

## Constant propagation

In [19]:
const constant_a = 1
function constant_propagation_example()
    a = constant_a
    b = a + 1
    return b > 1 ? 10 : 20
end
constant_propagation_example()
@code_typed constant_propagation_example()

CodeInfo(
1 ─     return 10
) => Int64

In [20]:
# If it's a variable, then all bets are off...
variable_a = 1
function propagate_my_variable()
    a = variable_a
    b = a + 1
    return b > 1 ? 10 : 20
end
propagate_my_variable()
@code_typed propagate_my_variable()

CodeInfo(
1 ─ %1 = Main.variable_a::Any
│   %2 = (%1 + 1)::Any
│   %3 = (%2 > 1)::Any
└──      goto #3 if not %3
2 ─      return 10
3 ─      return 20
) => Int64

Because functions are specialized by argument types. It solves the performance issue.

In [23]:
# using global variable
variable = 10
function add_by_passing_global_variable(x, v)
    return x + v
end

@btime add_by_passing_global_variable(10, $variable);

  2.591 ns (0 allocations: 0 bytes)


In [24]:
const constant = 10
function add_by_passing_global_constant(x, v)
    return x + v
end
@btime add_by_passing_global_constant(10, $constant);

  2.851 ns (0 allocations: 0 bytes)


In [29]:
variable = 10
function sum_variable_many_times(n)
    total = rand(variable)
    for i in 1:n
        total .+= rand(variable)
    end
    return total
end
@btime sum_variable_many_times(100);

  59.306 μs (301 allocations: 20.45 KiB)


In [31]:
const constant = 10
function sum_constant_many_times(n)
    total = rand(constant)
    for i in 1:n
        total .+= rand(constant)
    end
    return total
end
@btime sum_constant_many_times(100);


  11.105 μs (101 allocations: 14.20 KiB)
